In [7]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tensornetworks_pytorch.TNModels import PosMPS, Born
print(torch.__version__)

1.8.0.dev20201128


Use some data

In [27]:
import pickle
for dataset in [#'biofam',
    'flare','lymphography','spect','tumor','votes']:
    with open('datasets/'+dataset, 'rb') as f:
            a=pickle.load(f)
    X=a[0].astype(int)
    print(dataset)
    print("\tdata shape:", X.shape)
    print(f"\trange of X values: {X.min()} -- {X.max()}")

def load_dataset(dataset):
    with open('datasets/flare', 'rb') as f:
            a=pickle.load(f)
    X=a[0]
    X=X.astype(int)

    print("data shape:", X.shape)
    print(f"range of X values: {X.min()} -- {X.max()}")
    print(f"setting d={X.max()+1}")
    d = X.max()+1
    return X, d

flare
	data shape: (1065, 13)
	range of X values: 0 -- 7
lymphography
	data shape: (148, 19)
	range of X values: 0 -- 7
spect
	data shape: (187, 23)
	range of X values: 0 -- 1
tumor
	data shape: (339, 17)
	range of X values: 0 -- 3
votes
	data shape: (435, 17)
	range of X values: 0 -- 2


In [28]:
X,d = load_dataset('flare')

data shape: (1065, 13)
range of X values: 0 -- 7
setting d=8


In [29]:
D = 5
mps = PosMPS(D=D, d=d, verbose=True)
rBorn = Born(D=D, d=d, dtype=torch.float, verbose=True) 
cBorn = Born(D=D, d=d, dtype=torch.cfloat, verbose=True)
models = (mps, rBorn, cBorn)
for model in models:
    print(model.core.shape, model.name)
print("===")
for model in models:
    model.fit(X, d)

torch.Size([8, 5, 5]) Positive MPS
torch.Size([8, 5, 5]) Born model torch.float32
torch.Size([8, 5, 5]) Born model torch.complex64
===


In [31]:
batchsize=25
trainloader = DataLoader(X, batch_size=batchsize, shuffle=True)

model = mps
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

for batch_idx, batch in enumerate(trainloader):
    model.zero_grad()
    neglogprob = 0
    for x in batch:
        neglogprob -= model(x)
    loss = neglogprob / batchsize
    loss.backward()
    with torch.no_grad():
        print(batch_idx, "\tavg neglogprob:", loss)
    optimizer.step()

0 	avg neglogprob: tensor(25.9563, grad_fn=<DivBackward0>)
1 	avg neglogprob: tensor(23.4080, grad_fn=<DivBackward0>)
2 	avg neglogprob: tensor(22.1375, grad_fn=<DivBackward0>)
3 	avg neglogprob: tensor(20.3562, grad_fn=<DivBackward0>)
4 	avg neglogprob: tensor(19.9637, grad_fn=<DivBackward0>)
5 	avg neglogprob: tensor(18.7402, grad_fn=<DivBackward0>)
6 	avg neglogprob: tensor(17.6213, grad_fn=<DivBackward0>)
7 	avg neglogprob: tensor(17.1146, grad_fn=<DivBackward0>)
8 	avg neglogprob: tensor(17.0139, grad_fn=<DivBackward0>)
9 	avg neglogprob: tensor(17.2552, grad_fn=<DivBackward0>)
10 	avg neglogprob: tensor(16.6378, grad_fn=<DivBackward0>)
11 	avg neglogprob: tensor(16.1905, grad_fn=<DivBackward0>)
12 	avg neglogprob: tensor(16.2000, grad_fn=<DivBackward0>)
13 	avg neglogprob: tensor(16.2584, grad_fn=<DivBackward0>)
14 	avg neglogprob: tensor(15.7872, grad_fn=<DivBackward0>)
15 	avg neglogprob: tensor(15.6444, grad_fn=<DivBackward0>)
16 	avg neglogprob: tensor(15.5446, grad_fn=<DivBa